In [1]:
import pandas as pd
from IPython.display import Image, HTML
from datetime import datetime

In [2]:
# convert link to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

In [3]:
# Grab raw data to convert to html pages.
dfs = pd.read_csv("inputs/dfs.csv")
fd = pd.read_csv("inputs/fanduel.csv")
dk = pd.read_csv("inputs/draftkings.csv")
st = pd.read_csv("inputs/statline.csv")
dfs = dfs.drop("Unnamed: 0", axis=1)
fd = fd.drop("Unnamed: 0", axis=1)
dk = dk.drop("Unnamed: 0", axis=1)
st = st.drop("Unnamed: 0", axis=1)

In [4]:
# Grab timestamp.
ts = datetime.now()
ts = ts.strftime("%b %d, %Y %I:%M %p")
ts = "Last Updated: " + ts

In [5]:
# Highlights given player row to indicate hot/cold status for 19 columns.
def highlight_dfs(df):
    if df.Hot > 0.0 and df.Hot < 0.5: return ['background-color: #ff9999']*19
    elif df.Hot >= 0.5 and df.Hot < 1.0: return ['background-color: #ff6666']*19
    elif df.Hot >= 1.0 and df.Hot < 1.5: return ['background-color: #ff3333']*19
    elif df.Hot >= 1.5: return ['background-color: #ff0000']*19
    elif df.Cold > 0.0 and df.Cold < 0.5: return ['background-color: #99e6ff']*19
    elif df.Cold >= 0.5 and df.Cold < 1.0: return ['background-color: #66d9ff']*19
    elif df.Cold >= 1.0 and df.Cold < 1.5: return ['background-color: #33ccff']*19
    elif df.Cold >= 1.5: return ['background-color: #00bfff']*19
    else: return ['background-color: white']*19

In [6]:
# Highlights given player row to indicate hot/cold status for 14 columns.
def highlight_fd(df):
    if df.Hot > 0.0 and df.Hot < 0.5: return ['background-color: #ff9999']*14
    elif df.Hot >= 0.5 and df.Hot < 1.0: return ['background-color: #ff6666']*14
    elif df.Hot >= 1.0 and df.Hot < 1.5: return ['background-color: #ff3333']*14
    elif df.Hot >= 1.5: return ['background-color: #ff0000']*14
    elif df.Cold > 0.0 and df.Cold < 0.5: return ['background-color: #99e6ff']*14
    elif df.Cold >= 0.5 and df.Cold < 1.0: return ['background-color: #66d9ff']*14
    elif df.Cold >= 1.0 and df.Cold < 1.5: return ['background-color: #33ccff']*14
    elif df.Cold >= 1.5: return ['background-color: #00bfff']*14
    else: return ['background-color: white']*14

In [7]:
# Adds fire or ice emoji for respective hot/cold status.
def emojize(df):
    for i in range(len(df)):
        if df['Hot'][i+1] > 0.0: df['Name'][i+1] = df['Name'][i+1] + ('\n🔥')
        if df['Hot'][i+1] >= 1.0: df['Name'][i+1] = df['Name'][i+1] + ('🔥')
        if df['Hot'][i+1] >= 2.0: df['Name'][i+1] = df['Name'][i+1] + ('🔥')
        if df['Cold'][i+1] > 0.0: df['Name'][i+1] = df['Name'][i+1] + ('\n❄️')
        if df['Cold'][i+1] >= 1.0: df['Name'][i+1] = df['Name'][i+1] + ('❄️')
        if df['Cold'][i+1] >= 2.0: df['Name'][i+1] = df['Name'][i+1] + ('❄️')
    return df

In [8]:
# Error Handling

# Players that don't have a BBall Ref picture, as of 8/7/20.
no_pic_list = ["Paul Watson", "Donta Hall"]

# Player names that don't match to our in-house data.
special_names =       {"Luka Dončić": "Luka Doncic", 
                       "Luka Šamanić": "Luka Samanic", 
                       "Kristaps Porziņģis": "Kristaps Porzingis", 
                       "Nikola Vučević": "Nikola Vucevic",
                       "Jonas Valančiūnas": "Jonas Valanciunas",
                       "Bogdan Bogdanović": "Bogdan Bogdanovic",
                       "Dario Šarić": "Dario Saric",
                       "Timothé Luwawu-Cabarrot": "Timothe Luwawu-Cabarrot",
                       "Džanan Musa": "Dzanan Musa",
                        "Dāvis Bertāns": "Davis Bertans",
                        "Boban Marjanović": "Boban Marjanovic",
                        "Ersan İlyasova": "Ersan Ilyasova",
                        "Anžejs Pasečņiks": "Anzejs Pasecniks",
                       "Bojan Bogdanović": "Bojan Bogdanovic",
                        "Nicolò Melli": "Nicolo Melli",
                        "Nikola Jokić": "Nikola Jokic",
                        "Jusuf Nurkić": "Jusuf Nurkic",
                        "Goran Dragić": "Goran Dragic",
                        "Dennis Schröder" :"Dennis Schroder",
                       "Gary Payton": "Gary Payton II",
                       "Mohamed Bamba": "Mo Bamba",
                       "Wesley Iwundu": "Wes Iwundu",
                        "J.J. Redick": "JJ Redick",
                        "B.J. Johnson": "BJ Johnson"}
# Invert dictionary for use in data grabbing.
inv_names = {v: k for k, v in special_names.items()}

inv_names

{'Luka Doncic': 'Luka Dončić',
 'Luka Samanic': 'Luka Šamanić',
 'Kristaps Porzingis': 'Kristaps Porziņģis',
 'Nikola Vucevic': 'Nikola Vučević',
 'Jonas Valanciunas': 'Jonas Valančiūnas',
 'Bogdan Bogdanovic': 'Bogdan Bogdanović',
 'Dario Saric': 'Dario Šarić',
 'Timothe Luwawu-Cabarrot': 'Timothé Luwawu-Cabarrot',
 'Dzanan Musa': 'Džanan Musa',
 'Davis Bertans': 'Dāvis Bertāns',
 'Boban Marjanovic': 'Boban Marjanović',
 'Ersan Ilyasova': 'Ersan İlyasova',
 'Anzejs Pasecniks': 'Anžejs Pasečņiks',
 'Bojan Bogdanovic': 'Bojan Bogdanović',
 'Nicolo Melli': 'Nicolò Melli',
 'Nikola Jokic': 'Nikola Jokić',
 'Jusuf Nurkic': 'Jusuf Nurkić',
 'Goran Dragic': 'Goran Dragić',
 'Dennis Schroder': 'Dennis Schröder',
 'Gary Payton II': 'Gary Payton',
 'Mo Bamba': 'Mohamed Bamba',
 'Wes Iwundu': 'Wesley Iwundu',
 'JJ Redick': 'J.J. Redick',
 'BJ Johnson': 'B.J. Johnson'}

In [9]:
# Creates an HTML page based on input DataFrame and respective coloring.
'''
Args: 
    df: input DataFrame
    out: name of output HTML file (refer to main pages for correct naming)
    coloring: hot/cold highlights for HTML page
'''
def gen_page(df, out, coloring):
    # Read games file to grab slug code, for BBall Ref data.
    games = pd.read_csv('games.csv').loc[:,['name','slug']]
    trans_games = games.set_index('name').T
    img_dict = trans_games.to_dict('list')
    
    # Update data in Name and Picture column to display link to BBall Ref page and profile picture, respectively.
    images = []
    names = []
    for i in df["Name"]:
        display_name = i
        if i in inv_names:
            display_name = inv_names[i]
        code = "https://raw.githubusercontent.com/BerkeleySportsAnalytics/sagb-site/master/images/profile_picture.jpg"
        # Catch errors when player is not tracked in games file.
        if img_dict.get(display_name) != None:
            if display_name not in no_pic_list:
                # BBall Ref image hyperlink.
                code = "https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/" + img_dict.get(display_name)[0] + ".jpg"
            # BBall Ref player profile hyperlink and embed code.
            display_name = "<a href=\"https://www.basketball-reference.com/players/" + \
                img_dict.get(display_name)[0][0] + "/" + img_dict.get(display_name)[0] + ".html\" target=\"_blank\">" + display_name + "</a>"
        images.append(code)
        names.append(display_name)
    df["Picture"] = images
    df["Name"] = names
    
    # Change the picture column to the second column in order.
    cols = df.columns.tolist()
    cols = [cols[0]] + cols[-1:] + cols[1:-1]
    df = df[cols]
    pd.set_option('display.max_colwidth', -1)
    df.index +=1
    
    # Header of output HTML file.
    pre = (
        "<head>\n<script src=\"sorttable.js\"></script>\n"
        "<link rel=\"stylesheet\" href=\"df_styles.css\">\n</head>\n"
    )
    pre = pre + ts + "\n"
    pre += (
        "\n<body>\n<div id=\"filter\">\n<input type=\"text\" id=\"name\" onkeyup=\"nameSearch()\" placeholder=\"Search for name...\">\n"
        "<input type=\"text\" id=\"team\" onkeyup=\"teamSearch()\" placeholder=\"Search for team...\">\n"
        "<label for=\"pos\">Search by position:</label>\n"
        "<select id=\"pos\" onchange=\"posSearch()\" class='form-control'>\n"
        "<option></option>\n"
        "<option>PG</option>\n"
        "<option>SG</option>\n"
        "<option>SF</option>\n"
        "<option>PF</option>\n"
        "<option>C</option>\n"
        "</select>\n"
        "</div>\n\n"
    )
    
    # Footer of output HTML file.
    post = (
        "\n"
        "\n<script src=\"main.js\"></script>"
        "\n</body>"
        )
    
    # Render dataframe as HTML.
    if coloring is not None:
        images = ['<img src="' + image + '" width="60">' for image in images]
        df["Picture"] = images
        df = emojize(df)
        body = df.style.apply(coloring, axis=1).hide_columns(['Hot', 'Cold']).render()
    else:
        html = HTML(df.to_html(escape=False ,formatters=dict(Picture=path_to_image_html)))
        body = html.data

    # Generate HTML and add configuration for output tables, so JavaScript functions can run properly.
    body = body.replace("<table id=", "<table ignore=", 1)
    body = body.replace("<table ", "<table id=\"tbl\" class=\"sortable\" ", 1)
    a = pre + body + post
    # write html to file
    with open(out, 'w', encoding="utf-8") as f:
        f.write(a)
    return df

In [10]:
# Create the page for given output.
gen_page(dfs, 'dfs.html', highlight_dfs)
gen_page(fd, 'fd.html', highlight_fd)
gen_page(dk, 'dk.html', highlight_fd)
gen_page(st, 'st.html', None)

<ipython-input-9-1d3efe937277>:12: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  img_dict = trans_games.to_dict('list')
<ipython-input-9-1d3efe937277>:39: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)
<ipython-input-7-4925ea65e2bc>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if df['Cold'][i+1] > 0.0: df['Name'][i+1] = df['Name'][i+1] + ('\n❄️')
<ipython-input-7-4925ea65e2bc>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Name,Picture,Position,Game,Team,Minutes,2PT FG,3PT FG,FTM,Rebounds,Assists,Blocks,Steals,Turnovers,Opponent Defensive Rank vs Position,Injury Indicator,Injury Details
1,"<a href=""https://www.basketball-reference.com/players/a/antetgi01.html"" target=""_blank"">Giannis Antetokounmpo</a>",https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/antetgi01.jpg,SF,TOR@MIL,MIL,30.55,9.28,1.08,4.78,12.78,5.10,1.33,0.69,3.71,4.0,GTD,Mouth
2,"<a href=""https://www.basketball-reference.com/players/d/doncilu01.html"" target=""_blank"">Luka Dončić</a>",https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/doncilu01.jpg,SG,DAL@UTA,DAL,38.18,6.35,3.81,6.32,9.80,10.97,0.25,0.83,4.99,15.0,O,Ankle
3,"<a href=""https://www.basketball-reference.com/players/j/jamesle01.html"" target=""_blank"">LeBron James</a>",https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/jamesle01.jpg,SF,DEN@LAL,LAL,42.37,8.09,3.22,5.90,10.54,11.31,0.95,1.89,4.07,11.0,GTD,Groin
4,"<a href=""https://www.basketball-reference.com/players/d/davisan02.html"" target=""_blank"">Anthony Davis</a>",https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/davisan02.jpg,PF,DEN@LAL,LAL,41.98,9.38,0.87,3.67,8.38,0.95,1.66,1.92,2.05,6.0,GTD,Ankle
5,"<a href=""https://www.basketball-reference.com/players/p/porzikr01.html"" target=""_blank"">Kristaps Porziņģis</a>",https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/porzikr01.jpg,PF,DAL@UTA,DAL,37.70,5.73,2.08,3.62,11.28,1.40,1.30,0.66,2.84,13.0,O,Knee
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,"<a href=""https://www.basketball-reference.com/players/j/johnsal02.html"" target=""_blank"">Alize Johnson</a>",https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/johnsal02.jpg,PF,IND@MIA,IND,12.54,0.69,0.00,0.17,1.19,0.00,0.55,0.22,0.00,6.0,,
162,"<a href=""https://www.basketball-reference.com/players/a/antetko01.html"" target=""_blank"">Kostas Antetokounmpo</a>",https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/antetko01.jpg,PF,DEN@LAL,LAL,13.29,0.00,0.29,0.00,0.76,0.93,0.42,0.79,0.13,6.0,,
163,"<a href=""https://www.basketball-reference.com/players/o/onimi01.html"" target=""_blank"">Miye Oni</a>",https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/onimi01.jpg,SG,DAL@UTA,UTA,14.57,0.00,0.12,0.00,1.36,0.00,0.25,0.74,0.21,6.0,,
164,"<a href=""https://www.basketball-reference.com/players/w/wrighju02.html"" target=""_blank"">Justin Wright-Foreman</a>",https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/wrighju02.jpg,PG,DAL@UTA,UTA,14.58,0.00,0.10,0.00,1.85,0.14,0.02,0.98,0.98,9.0,,
